In [1]:
import numpy as np
import pandas as pd
import os

from pymongo import MongoClient
import pymongo

import matplotlib.pyplot as plt 
import matplotlib
import re
import jieba
import jieba.analyse
import jieba.posseg as pseg

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download("punkt") # 下載需要用到的語料庫
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import fasttext

[nltk_data] Downloading package punkt to /home/evalogin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/evalogin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/evalogin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/evalogin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"
! nvidia-smi

Thu Jun 23 15:17:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 34%   45C    P2    80W / 260W |   3044MiB / 11019MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 26%   

# 取得測試資料

In [3]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn
    
    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de
    
    db_rm_us_2020 = db.rm_patent_us_2020
    db_rm_cn_2020 = db.rm_patent_cn_2020
    db_rm_de_2020 = db.rm_patent_de_2020
    
except errors.ConnectionFailure as err:
    print(err)

In [4]:
data_cn_desc = db_rm_cn_2020.find({}, {"patentNumber":1, "description": 1})

ls_id = []
ls_pNo = []
ls_desc = []
for data in data_cn_desc:
    # print(data["_id"],data["count"])
    ls_id.append(data["_id"])
    ls_pNo.append(data["patentNumber"])
    ls_desc.append(data["description"])

In [5]:
df_cn = pd.DataFrame((zip(ls_id, ls_pNo, ls_desc)), columns = ['id', 'patentNumber', 'description'])

In [6]:
# df_cn.head()

In [7]:
df_cn

,id,patentNumber,description
0,61ffe131f4d57576a502c4bf,CN112187940A,技术领域本发明涉及通信技术领域，尤其涉及到一种基于物联网的安防监控系统。背景技术随着通信、微...
1,61ffe143f4d57576a502c4d3,CN112364317A,技术领域本发明涉及物联网信息安全技术领域，具体涉及一种基于区块链技术的物联网雾环境管理架构及...
2,61ffe14df4d57576a502c4d5,CN113508066A,相关申请的交叉引用本申请要求于2019年3月29日提交的题为自主交通工具系统(“Autono...
3,61ffe14ef4d57576a502c4d7,CN213399967U,技术领域本实用新型属于城市公共设施技术领域，具体涉及一种城市公共设施资产管理系统。背景技术目...
4,61ffe181f4d57576a502c50b,CN111382190B,技术领域本发明涉及人工智能技术领域，具体涉及一种基于智能的对象推荐方法、装置和存储介质。背景...
...,...,...,...
7687,620072d9f4d57576a50387db,CN111935889A,技术领域本发明涉及自动采集及远程控制领域，具体是一种路灯自动采集及远程控制装置。背景技术随着...
7688,620072d9f4d57576a50387dd,CN212660357U,技术领域本发明涉及自动采集及远程控制领域，具体是一种路灯自动采集及远程控制装置。背景技术随着...
7689,620072dff4d57576a50387eb,CN111353035B,技术领域本公开涉及计算机技术领域，具体地，涉及一种人机对话方法、装置、可读存储介质及电子设备...
7690,620072f3f4d57576a5038803,CN211184794U,技术领域本实用新型涉及一种电子装置的散热模组，尤其涉及一种具有鳍片组的连接器。背景技术由于5...


## 資料處理

### 中文

In [8]:
df_cn.description.isna().sum()

0

In [9]:
df_cn.description.fillna("", inplace=True)

In [10]:
def clean(row):
    text = re.sub('[^\u4e00-\u9fa5；、，。 ]', '', row) #除了文字,其他數字空白?!.的都去掉
    return text

In [11]:
df_cn["description"] = df_cn.description.apply(clean)

In [12]:
# row = "用于传感器驱动的车辆遥测应用和服务的开发和部署的计算 平台技术领域本发明通常涉及智能泛在计算系统、信息物理系统以及物联网领域。更具体 地，本发明涉及用于允许这样一种统一平台的方法和系统，该统一平台能够对于智能泛在 交通环境中的传感器驱动的车辆遥测应用的开发和部署提供服务套件。背景技术通常，智能车辆交通包括使用远距离通信和遥测数据分析来远程地监视车辆、路 况、远程车辆监控，和驾驶条件等。远距离通信已经在世界范围内广泛实现以用于分析由远 程目标"
# sent = re.split('; |。|，', row)
# token = [jieba.lcut(i) for i in sent]

In [13]:
# ls = []
# for w in token:
#     # ls2 = [i for i in w if len(i)>1 and nltk.pos_tag([i])]
#     ls2 = [i for i in w if len(i)>1 and (nltk.pos_tag([i])[0][1][0]=="N" or nltk.pos_tag([i])[0][1][0]=="V")]

In [14]:
# ls2

In [15]:
# token[2][0]

In [16]:
# nltk.pos_tag([token[2][0]])

In [17]:
#新增自定義字典
jieba.load_userdict('./dictionary_com.text')
#新增stop words
with open('./stopwords.txt',encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]


# 自定義斷詞 function
def getToken(row):
    # seg_list = jieba.lcut(row) # jieba斷詞
    # # seg_list = [w for w in seg_list if w not in stopWords and len(w) > 1] # 篩選不在停用字的字與字元數大於1的字詞
    # seg_list = [w for w in seg_list if len(w) > 1]
    
    sent = re.split('; |。|，', row)
    token = [jieba.lcut(i) for i in sent]

    ls = []
    for w in token:
        for i in w :
            if i not in stopWords and len(i)>1 and (nltk.pos_tag([i])[0][1][0]=="N" or nltk.pos_tag([i])[0][1][0]=="V"):
                ls.append(i)
    
    return ls

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.491 seconds.
Prefix dict has been built successfully.


In [18]:
# row = df_cn2.description[0]

In [19]:
# df_cn2 = df_cn.iloc[0:10,:]

In [20]:
# df_cn2

In [21]:
# df_cn2["word"] = df_cn2.description.apply(getToken)

In [22]:
# df_cn2

In [23]:
df_cn["word"] = df_cn.description.apply(getToken)

In [24]:
df_cn.head()

,id,patentNumber,description,word
0,61ffe131f4d57576a502c4bf,CN112187940A,技术领域本发明涉及通信技术领域，尤其涉及到一种基于物联网的安防监控系统。背景技术随着通信、微...,"[技术, 领域, 本发明, 涉及, 通信, 技术, 领域, 涉及, 一种, 基于, 联网, ..."
1,61ffe143f4d57576a502c4d3,CN112364317A,技术领域本发明涉及物联网信息安全技术领域，具体涉及一种基于区块链技术的物联网雾环境管理架构及...,"[技术, 领域, 本发明, 涉及, 联网, 信息安全, 技术, 领域, 具体, 涉及, 一种..."
2,61ffe14df4d57576a502c4d5,CN113508066A,相关申请的交叉引用本申请要求于年月日提交的题为自主交通工具系统 的美国临时专利申请第号的权益...,"[相关, 申请, 交叉, 引用, 申请, 提交, 题为, 自主, 交通工具, 系统, 美国,..."
3,61ffe14ef4d57576a502c4d7,CN213399967U,技术领域本实用新型属于城市公共设施技术领域，具体涉及一种城市公共设施资产管理系统。背景技术目...,"[技术, 领域, 实用新型, 属于, 城市, 公共设施, 技术, 领域, 具体, 涉及, 一..."
4,61ffe181f4d57576a502c50b,CN111382190B,技术领域本发明涉及人工智能技术领域，具体涉及一种基于智能的对象推荐方法、装置和存储介质。背景...,"[技术, 领域, 本发明, 涉及, 人工智能, 技术, 领域, 具体, 涉及, 一种, 基于..."


In [25]:
# df_cn_token = df_cn.explode('word') # 將word欄位展開
df_cn_token = df_cn.copy()

In [26]:
df_cn_token.dropna(subset = ["word"], inplace=True)
df_cn_token.reset_index(inplace=True, drop=True) # 原本index基於文章筆數，現在需要重設index為基於斷詞數量
df_cn_token.head()

,id,patentNumber,description,word
0,61ffe131f4d57576a502c4bf,CN112187940A,技术领域本发明涉及通信技术领域，尤其涉及到一种基于物联网的安防监控系统。背景技术随着通信、微...,"[技术, 领域, 本发明, 涉及, 通信, 技术, 领域, 涉及, 一种, 基于, 联网, ..."
1,61ffe143f4d57576a502c4d3,CN112364317A,技术领域本发明涉及物联网信息安全技术领域，具体涉及一种基于区块链技术的物联网雾环境管理架构及...,"[技术, 领域, 本发明, 涉及, 联网, 信息安全, 技术, 领域, 具体, 涉及, 一种..."
2,61ffe14df4d57576a502c4d5,CN113508066A,相关申请的交叉引用本申请要求于年月日提交的题为自主交通工具系统 的美国临时专利申请第号的权益...,"[相关, 申请, 交叉, 引用, 申请, 提交, 题为, 自主, 交通工具, 系统, 美国,..."
3,61ffe14ef4d57576a502c4d7,CN213399967U,技术领域本实用新型属于城市公共设施技术领域，具体涉及一种城市公共设施资产管理系统。背景技术目...,"[技术, 领域, 实用新型, 属于, 城市, 公共设施, 技术, 领域, 具体, 涉及, 一..."
4,61ffe181f4d57576a502c50b,CN111382190B,技术领域本发明涉及人工智能技术领域，具体涉及一种基于智能的对象推荐方法、装置和存储介质。背景...,"[技术, 领域, 本发明, 涉及, 人工智能, 技术, 领域, 具体, 涉及, 一种, 基于..."


In [27]:
df_cn_token.head(20)

,id,patentNumber,description,word
0,61ffe131f4d57576a502c4bf,CN112187940A,技术领域本发明涉及通信技术领域，尤其涉及到一种基于物联网的安防监控系统。背景技术随着通信、微...,"[技术, 领域, 本发明, 涉及, 通信, 技术, 领域, 涉及, 一种, 基于, 联网, ..."
1,61ffe143f4d57576a502c4d3,CN112364317A,技术领域本发明涉及物联网信息安全技术领域，具体涉及一种基于区块链技术的物联网雾环境管理架构及...,"[技术, 领域, 本发明, 涉及, 联网, 信息安全, 技术, 领域, 具体, 涉及, 一种..."
2,61ffe14df4d57576a502c4d5,CN113508066A,相关申请的交叉引用本申请要求于年月日提交的题为自主交通工具系统 的美国临时专利申请第号的权益...,"[相关, 申请, 交叉, 引用, 申请, 提交, 题为, 自主, 交通工具, 系统, 美国,..."
3,61ffe14ef4d57576a502c4d7,CN213399967U,技术领域本实用新型属于城市公共设施技术领域，具体涉及一种城市公共设施资产管理系统。背景技术目...,"[技术, 领域, 实用新型, 属于, 城市, 公共设施, 技术, 领域, 具体, 涉及, 一..."
4,61ffe181f4d57576a502c50b,CN111382190B,技术领域本发明涉及人工智能技术领域，具体涉及一种基于智能的对象推荐方法、装置和存储介质。背景...,"[技术, 领域, 本发明, 涉及, 人工智能, 技术, 领域, 具体, 涉及, 一种, 基于..."
5,61ffe183f4d57576a502c50d,CN111510433A,技术领域本发明涉及一种基于雾计算平台的物联网恶意流量检测方法，属于网络安全技术领域。背景技术...,"[技术, 领域, 本发明, 涉及, 一种, 基于, 计算, 平台, 联网, 恶意, 流量, ..."
6,61ffe196f4d57576a502c527,CN112488398A,技术领域本发明涉及电力物联网边缘计算的技术领域，尤其涉及一种基于边缘智能网关的用电管理方法及...,"[技术, 领域, 本发明, 涉及, 电力, 联网, 边缘, 计算, 技术, 领域, 涉及, ..."
7,61ffe1ebf4d57576a502c545,CN111595377A,技术领域本发明涉及导弹贮运技术领域，具体来说，涉及一种导弹贮运环境监测系统。背景技术随着我军...,"[技术, 领域, 本发明, 涉及, 导弹, 贮运, 技术, 领域, 具体来说, 涉及, 一种..."
8,61ffe1ecf4d57576a502c547,CN111405050A,技术领域本发明涉及智能监测设备领域，特别是一种基于区块链技术的智能监测平台。背景技术目前在物...,"[技术, 领域, 本发明, 涉及, 智能, 监测, 设备, 领域, 特别, 一种, 基于, ..."
9,61ffe1fbf4d57576a502c555,CN112508729A,技术领域本发明涉及人工智能技术领域，尤其涉及一种智能乡村产业数字化应用管理方法、装置、设备及...,"[技术, 领域, 本发明, 涉及, 人工智能, 技术, 领域, 涉及, 一种, 智能, 乡村..."


## 存成txt檔

In [28]:
# ! head -c 100 data/fil9

In [29]:
# 存成txt檔
textfile = open("./data/cn_file_description_paragraph_2020.txt", "w")
for i in range(len(df_cn_token)):
    for element in df_cn_token.word[i]:
        # print(element)
        textfile.write(element+" ")
    textfile.write("\n")
textfile.close()